<a href="https://colab.research.google.com/github/ZainAli24/Quater_04/blob/main/OpenRouter_onsite_class01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. OpenRouter: A Unified Interface for 50 Free LLMs :**

Haan, aapne OpenRouter ka concept kaafi had tak sahi samjha hai. Main aapki baat ko summarize aur thodi clarity ke saath explain karta hoon:

1. **LangChain se Similarity**:  
   Aapne bilkul sahi kaha ke OpenRouter ka kaam LangChain se milta-julta hai. LangChain mein hum ek unified interface use karte hain jahan sirf model name (jaise GPT-4, Claude, ya koi aur) aur API key provide karni hoti hai, aur baki ka code same rehta hai, model badalne par code change nahi karna padta. OpenRouter bhi isi tarah ka unified interface provide karta hai, jahan aapko bas model name batana hota hai (jaise Gemini, o1, ya koi aur), aur aapka code model-agnostic rehta hai.

2. **OpenRouter ka Key Benefit - Single API Key**:  
   Aapka yeh point bhi sahi hai ke OpenRouter ka ek bada faida yeh hai ke aapko har model provider (jaise OpenAI, Anthropic, Google) ke alag-alag API keys use karne ki zarurat nahi. Aap sirf OpenRouter ka ek API key use karte ho, aur usi key ke saath aap OpenRouter ke platform par available koi bhi model select kar sakte ho. Yeh developers ke liye kaam ko bohot simplify karta hai kyunki alag-alag providers ke keys manage karne ka jhanjhat khatam ho jata hai.

3. **OpenAI-Compatible API**:  
   Aapne yeh bhi mention kiya ke OpenRouter OpenAI ke chat completion API ke saath compatible hai, aur yeh bilkul sahi hai. OpenRouter ne apna API is tarah design kiya hai ke woh OpenAI ke API structure ko follow karta hai. Is wajah se agar aap pehle se OpenAI ke API calls ka code use kar rahe ho, toh OpenRouter ke saath integrate karna bohot asaan ho jata hai. Bas API key aur base URL (`https://openrouter.ai/api/v1`) change karna hota hai, aur model name specify karna hota hai.

4. **Model Selection aur Flexibility**:  
   OpenRouter ka ek aur faida yeh hai ke ismein aap bohot saare models (open-source aur closed-source dono) ek hi platform par access kar sakte ho. Aapko bas model name batana hota hai (jaise `mistralai/mixtral-8x7b` ya `anthropic/claude-3.5-sonnet`), aur OpenRouter us model ke saath request ko route kar deta hai. Isse aapko alag-alag providers ke APIs ke technical details mein nahi ulajhna padta.

### Thodi Aur Clarity:
- **API Key ki Baat**: Aapne kaha ke model provider ka API key nahi dena padta, yeh sahi hai, lekin OpenRouter ka apna API key dena zaroori hota hai. Yeh key aap OpenRouter ke website se generate karte ho, aur yeh ek hi key kaafi hoti hai sab models ke liye.
- **Cost aur Pricing**: OpenRouter ka ek aur faida yeh hai ke yeh pay-per-token pricing model follow karta hai, aur open-source models ke liye kaafi low-cost hota hai compared to proprietary models. Isse aap apne project ke budget ke hisaab se models choose kar sakte ho.
- **Playground aur Testing**: OpenRouter ek playground bhi provide karta hai jahan aap different models ko test kar sakte ho aur unki performance, latency, aur cost compare kar sakte ho.

### Aapki Understanding:
Aapki understanding bilkul sahi hai ke OpenRouter LangChain ki tarah hi ek abstraction layer provide karta hai, jisse developers ko model switching aur integration mein flexibility milti hai. Yeh OpenAI ke API ke saath compatibility aur single API key ka concept OpenRouter ko bohot developer-friendly banata hai.


# **2. OpenRouters Rata Limits for Free Models :**

## **Rate Limits:**
### LLMs API rate limits are measured across several dimensions:
- ### Requests per minute (RPM)  
- ### Tokens per minute (TPM)  
- ### Requests per day (RPD)


## **OpenRouters Free Models Rata Limits:**
### OpenRouter’s free models typically have a global limit of **200 requests per day (RPD)** across all free models, with some also capped at **20 requests per minute (RPM)**, as noted in their documentation. These limits apply to models with IDs ending in :free.

# **3. for use OpenRouter with OpenAI Agents SDK we install openai agents sdk :**
```python
pip install openai-agents
```

In [13]:
!pip install -Uq openai-agents

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

# **4. OpneRouter ik middle man ki tarah kaam kar rha hai, jo hamari request ko jo specific model choose kia hai wa tak bhejta hai , toh is liye hume request direct us model provider pe nahe karni hume request opnerouter pe karni hai , toh is liye hum `base_url` mien opnerouter ka url denge:**
  ```python
  BASE_URL = "https://openrouter.ai/api/v1"
  ```

In [16]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.0-flash-exp:free"    # use free model.

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

# **Method 1. Using the OpenRouter API directly:**
## **1. OpenRouter ke saath API Request Kaise Bhejte Hain:**

OpenRouter ke saath API request bhejne ke liye hum `requests.post` method ka use karte hain. Is process mein hum teen main cheezein bhejte hain:

1. **Base URL**: Yeh woh endpoint hai jahan request bheji jati hai. OpenRouter ke liye base URL hota hai:  
   `https://openrouter.ai/api/v1/chat/completions`

2. **Authorization (API Key)**: Yeh OpenRouter ka API key hota hai, jo headers mein `Authorization` field ke zariye bheja jata hai. Yeh key batata hai ke user authenticated hai ya nahi.  
   Format: `"Authorization": f"Bearer {OPENROUTER_API_KEY}"`

3. **Data (JSON Format)**: Yeh woh payload hai jo hum request mein bhejte hain. Ismein do main cheezein hoti hain:
   - **Model**: Hum specify karte hain ke kaunsa model use karna hai (jaise `gpt-4`, `claude-3.5-sonnet`, ya `mistralai/mixtral-8x7b`).
   - **Messages**: Yeh ek list hoti hai jismein hum model ko bhejna wala message define karte hain. Har message mein `role` (jaise `user`) aur `content` (user ka query) hota hai.

### Code Example
Yeh raha aapka code, jo OpenRouter ke saath ek POST request bhejta hai:

```python
import requests
import json

# Constants
BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = "your_openrouter_api_key_here"  # Apna API key yahan daalein
MODEL = "anthropic/claude-3.5-sonnet"  # Yahan desired model ka naam daalein

# POST request bhejne ka code
response = requests.post(
    url=f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    },
    data=json.dumps({
        "model": MODEL,
        "messages": [
            {
                "role": "user",
                "content": "you know which car company is making auto drive"
            }
        ]
    })
)

# Response print karna
print(response.json())
```

### Code Explanation
- **`url`**: Yeh endpoint specify karta hai jahan request jati hai (`/chat/completions`).
- **`headers`**: Ismein `Authorization` key ke saath OpenRouter ka API key bheja jata hai.
- **`data`**: JSON format mein data bheja jata hai, jismein `model` aur `messages` define kiye jate hain.
- **`json.dumps`**: Python dictionary ko JSON string mein convert karta hai.
- **`response.json()`**: Server se aaya hua response JSON format mein print karta hai.

### Key Points
- **Unified Interface**: OpenRouter ka yeh API OpenAI ke chat completion API ke saath compatible hai, isliye agar aap OpenAI ka code use kar rahe hain, toh sirf base URL aur API key change karke OpenRouter use kar sakte hain.
- **Error Handling**: Production code mein aapko response status check karna chahiye (jaise `response.status_code == 200`) aur errors ko handle karna chahiye.
- **Model Flexibility**: Aap `MODEL` variable mein koi bhi OpenRouter-supported model daal sakte hain bina code change kiye.

---

### Improvements ke Suggestions
1. **Environment Variables**: `OPENROUTER_API_KEY` ko code mein hardcode karne ke bajaye, environment variable (jaise `.env` file) mein store karna better hai for security.
   ```python
   import os
   OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
   ```
2. **Error Handling**:
   ```python
   if response.status_code == 200:
       print(response.json())
   else:
       print(f"Error: {response.status_code} - {response.text}")
   ```
3. **Multiple Messages**: Aap `messages` list mein multiple messages daal sakte hain, jaise system prompt ya previous conversation:
   ```python
   "messages": [
       {"role": "system", "content": "You are a helpful assistant."},
       {"role": "user", "content": "Which car company is making auto drive?"}
   ]
   ```

---



In [22]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content":"you know which car company is making auto driving cars"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745476841-Ev61QX1JbT3GiD2dZIUW', 'provider': 'Google AI Studio', 'model': 'google/gemini-2.0-flash-exp', 'object': 'chat.completion', 'created': 1745476841, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': 'Many car companies are developing autonomous driving technology. Here are some of the major players:\n\n*   **Tesla:** Known for its Autopilot and Full Self-Driving (FSD) features, though these are currently classified as advanced driver-assistance systems (ADAS) and not full autonomy.\n*   **Waymo (owned by Alphabet, Google\'s parent company):** Focuses specifically on autonomous driving technology and operates a robotaxi service in some areas.\n*   **General Motors (Cruise):** Another company working on autonomous vehicles and operating a robotaxi service, though with some recent setbacks.\n*   **Ford:** Developing autonomous driving technology, often in partnership with oth

In [23]:
data = response.json()
print(data['choices'][0]['message']['content'])

Many car companies are developing autonomous driving technology. Here are some of the major players:

*   **Tesla:** Known for its Autopilot and Full Self-Driving (FSD) features, though these are currently classified as advanced driver-assistance systems (ADAS) and not full autonomy.
*   **Waymo (owned by Alphabet, Google's parent company):** Focuses specifically on autonomous driving technology and operates a robotaxi service in some areas.
*   **General Motors (Cruise):** Another company working on autonomous vehicles and operating a robotaxi service, though with some recent setbacks.
*   **Ford:** Developing autonomous driving technology, often in partnership with other companies.
*   **BMW:** Actively working on autonomous driving features for its future vehicles.
*   **Mercedes-Benz:** Offering advanced driver-assistance systems and working towards higher levels of autonomy.
*   **Hyundai/Kia:** Investing heavily in autonomous driving and related technologies.
*   **Stellantis (Ch

# **Method 2. Using OpenRouter with OpenAI Agents SDK:**

## **2. OpenAI Agents SDK ke Components aur OpenRouter Integration:**

Aapke code aur description ke basis par, yeh raha formatted explanation aur code ka breakdown:

#### 1. **`AsyncOpenAI` Class**
- **Purpose**: Yeh class OpenAI Agents SDK mein LLMs (Large Language Models) ke saath integration ke liye use hoti hai. Iska kaam hai API requests ko handle karna aur LLM provider (is case mein OpenRouter) ke saath communication karna.
- **Inputs**:
  - **`api_key`**: OpenRouter ka API key jo authentication ke liye use hota hai.
  - **`base_url`**: Yeh woh endpoint hai jahan requests bheji jati hain (OpenRouter ke liye: `https://openrouter.ai/api/v1`).
- **Role**: Yeh class OpenAI ke API structure ke saath compatible hai, isliye OpenRouter ke saath seamlessly kaam karta hai.

#### 2. **`Agent` Class**
- **Purpose**: Yeh class ek agent ko define karti hai jo specific tasks perform karta hai.
- **Inputs**:
  - **`name`**: Agent ka naam (jaise "Assistant").
  - **`instructions`**: Agent ke liye instructions ya rules (jaise "respond only in haikus").
  - **`model`**: Yeh batata hai ke agent kaunsa LLM model use karega (is case mein OpenRouter ka model).
- **Role**: Agent ka kaam hai user ke prompts ke basis par intelligent responses generate karna, aur yeh model ke saath integrate hota hai.

#### 3. **`OpenAIChatCompletionsModel` Class**
- **Purpose**: Yeh class specify karti hai ke kaunsa model use hoga aur kaunsa provider (OpenRouter) us model ko serve karega.
- **Inputs**:
  - **`model`**: Model ka naam (jaise `anthropic/claude-3.5-sonnet` ya `openai/gpt-4`).
  - **`openai_client`**: `AsyncOpenAI` class ka instance, jo API key aur base URL ke saath configure kiya gaya hai.
- **Role**: Yeh class model aur provider ke details ko encapsulate karti hai taaki agent ko model ke saath interact karna asaan ho.

#### 4. **`set_tracing_disabled` Function**
- **Purpose**: Yeh function tracing (agent ke actions ko track karna) ko enable ya disable karta hai.
- **Inputs**:
  - **`disabled`**: `True` ya `False`. Agar `True` hai, toh tracing band ho jati hai, yani agent ke actions ka detailed log ya tracking nahi hota.
- **Role**: Tracing development ke liye useful hoti hai jab aap agent ke har step ko debug karna chahte hain. Production mein tracing disable karna performance ke liye better hota hai.

#### 5. **`Runner` Class**
- **Purpose**: Yeh class agent ko ek specific prompt ke saath run karti hai aur uska output return karti hai.
- **Inputs**:
  - **`agent`**: Agent ka instance jo run hoga.
  - **`prompt`**: User ka input ya query (jaise "Tell me about recursion in programming").
- **Role**: Yeh class agent ke execution ko handle karti hai aur final output generate karti hai.

---

### Code Example
Yeh raha aapka code, thoda formatted aur comments ke saath:

```python
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled
import asyncio

# Constants
OPENROUTER_API_KEY = "your_openrouter_api_key_here"  # Apna OpenRouter API key yahan daalein
BASE_URL = "https://openrouter.ai/api/v1"  # OpenRouter ka base URL
MODEL = "anthropic/claude-3.5-sonnet"  # Desired model ka naam

# AsyncOpenAI client initialize karna
client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

# Tracing disable karna (optional, performance ke liye)
set_tracing_disabled(disabled=True)

# Main async function
async def main():
    # Agent define karna
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",  # Agent ke instructions
        model=OpenAIChatCompletionsModel(
            model=MODEL,
            openai_client=client
        ),
    )

    # Agent ko prompt ke saath run karna
    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )

    # Final output print karna
    print(result.final_output)

# Async function ko run karna
if __name__ == "__main__":
    asyncio.run(main())
```

---

### Code Explanation
1. **Imports**:
   - `AsyncOpenAI`: OpenAI SDK se import kiya gaya hai, jo API requests handle karta hai.
   - `Agent`, `OpenAIChatCompletionsModel`, `Runner`, `set_tracing_disabled`: Agents SDK ke components hain.

2. **Client Setup**:
   - `AsyncOpenAI` ka instance banaya gaya hai jismein OpenRouter ka API key aur base URL pass kiya gaya hai.
   - Yeh client model ke saath communication ke liye use hota hai.

3. **Tracing**:
   - `set_tracing_disabled(True)` se tracing disable ki gayi hai taaki agent ke actions ka log na bane (optional).

4. **Agent Definition**:
   - `Agent` class ka instance banaya gaya hai jismein:
     - `name`: Agent ka naam ("Assistant").
     - `instructions`: Agent ko bataya gaya hai ke woh sirf haikus mein respond kare.
     - `model`: `OpenAIChatCompletionsModel` ka instance pass kiya gaya hai, jismein model name aur client define hain.

5. **Runner**:
   - `Runner.run` method ke zariye agent ko ek prompt ke saath run kiya gaya hai.
   - Prompt: "Tell me about recursion in programming."
   - Output: Agent haiku format mein response dega, jo `result.final_output` mein store hota hai.

6. **Async Execution**:
   - `asyncio.run(main())` se async function ko execute kiya gaya hai kyunki `AsyncOpenAI` aur `Runner` async operations use karte hain.

---

### Key Points
- **OpenRouter Integration**: `AsyncOpenAI` ka use karke OpenRouter ke saath integration bohot asaan hai kyunki OpenRouter OpenAI ke API structure ko follow karta hai.
- **Flexibility**: Aap `MODEL` variable change karke koi bhi OpenRouter-supported model (jaise `mistralai/mixtral-8x7b`, `openai/gpt-4o`) use kar sakte hain bina code badle.
- **Tracing**: Development ke dauraan tracing enable (`set_tracing_disabled(False)`) karna helpful ho sakta hai taaki aap agent ke har step ko track kar saken.
- **Haiku Instruction**: Is example mein agent sirf haikus mein jawab dega, jo ek creative constraint hai.

---

### Improvements ke Suggestions
1. **Error Handling**:
   - API calls ya async operations mein errors ko handle karna zaroori hai.
   ```python
   try:
       result = await Runner.run(agent, "Tell me about recursion in programming.")
       print(result.final_output)
   except Exception as e:
       print(f"Error: {e}")
   ```

2. **Environment Variables**:
   - API key ko secure rakhne ke liye environment variable use karein:
   ```python
   import os
   OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
   ```

3. **Multiple Prompts**:
   - Agar aap multiple prompts ya conversation history handle karna chahte hain, toh `Agent` class ke `messages` parameter ko extend kar sakte hain.

4. **Logging**:
   - Tracing ke alawa, aap custom logging add kar sakte hain taaki agent ke inputs aur outputs ka record rakha ja sake.

---

### Output Example
Agar model `anthropic/claude-3.5-sonnet` hai aur instruction "You only respond in haikus" hai, toh prompt "Tell me about recursion in programming" ka output kuch aisa ho sakta hai:

```
Code calls itself back,
Solving tasks by breaking down,
Base case stops the stack.
```

---

Aapki understanding bilkul sahi hai, aur yeh code OpenRouter ke saath Agents SDK ko effectively use karta hai.


In [21]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You are my Assistant for any kind of query",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me AI Agents Developer salary in the start beginner level monthly",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

It's tricky to give you a precise salary for a beginner-level AI Agent Developer, as it can vary significantly based on several factors. Here's a breakdown of the key influencing elements and a general salary range:

**Factors Influencing Salary:**

*   **Location:** Salaries are heavily influenced by location. Major tech hubs like San Francisco, New York, Seattle, and London will generally have higher salaries compared to smaller cities or areas with a lower cost of living.
*   **Company Size and Type:** Large, well-funded tech companies (FAANG, etc.) typically pay more than smaller startups or non-tech companies.
*   **Educational Background and Skills:** A degree in Computer Science, Artificial Intelligence, Machine Learning, or a related field is almost essential. Strong programming skills (Python, etc.) and experience with relevant frameworks (TensorFlow, PyTorch, Langchain, etc.) will increase your value.
*   **Specific AI Agent Skills:** The demand for certain types of AI agents